In [1]:
import pandas as pd
#import xlwings as xw
#import xlsxwriter
import os
import pickle
import math

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')   
os.chdir("/content/gdrive/My Drive//MIE490: Capstone/Data")

Mounted at /content/gdrive


In [3]:
files = os.listdir("HP comp")
files_to_process = []
for file in files:
    if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
        files_to_process.append(file)

files_to_process


['W_HP_WC.xlsx',
 'W_HP_WorldJr.xlsx',
 'M_HP_WC.xlsx',
 'M_HP_WorldChamp.xlsx',
 'M_HP_NC.xlsx',
 'M_HP_WorldJr.xlsx',
 'W_HP_QL.xlsx',
 'M_HP_QL.xlsx',
 'W_HP_NAC.xlsx',
 'W_HP_NC.xlsx',
 'W_HP_WorldChamp.xlsx',
 'M_HP_NAC.xlsx']

In [4]:
## get all the ranks for each competition
file = 'M_HP_WC.xlsx'
xls = pd.ExcelFile('HP comp/'+file)
sheet_names = xls.sheet_names
# each sheet is one competition
for sheet in sheet_names:
        df = xls.parse(sheet,header = 0)
        for a in range (df.shape[0]):
            print(df.loc[a, 'Rank'])
            #print(df)
        break

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
nan


In [6]:
sheet_names.reverse()

In [7]:
sheet_names

['M_HP_WC_19950120',
 'M_HP_WC_19950203',
 'M_HP_WC_19950210',
 'M_HP_WC_19950216',
 'M_HP_WC_19950226',
 'M_HP_WC_19960118',
 'M_HP_WC_19960119',
 'M_HP_WC_19960212',
 'M_HP_WC_19960217',
 'M_HP_WC_19960225',
 'M_HP_WC_19960303',
 'M_HP_WC_19960309',
 'M_HP_WC_19960317',
 'M_HP_WC_19961214',
 'M_HP_WC_19961215',
 'M_HP_WC_19970119',
 'M_HP_WC_19970202',
 'M_HP_WC_19970208',
 'M_HP_WC_19970216',
 'M_HP_WC_19970302',
 'M_HP_WC_19970316',
 'M_HP_WC_19981123',
 'M_HP_WC_19981211',
 'M_HP_WC_19981216',
 'M_HP_WC_19990107',
 'M_HP_WC_19990131',
 'M_HP_WC_19990206',
 'M_HP_WC_19990214',
 'M_HP_WC_19990220',
 'M_HP_WC_19990313',
 'M_HP_WC_19991128',
 'M_HP_WC_19991212',
 'M_HP_WC_19991219',
 'M_HP_WC_20000115',
 'M_HP_WC_20000122',
 'M_HP_WC_20000128',
 'M_HP_WC_20000129',
 'M_HP_WC_20000219',
 'M_HP_WC_20000227',
 'M_HP_WC_20000304',
 'M_HP_WC_20000311',
 'M_HP_WC_20000318',
 'M_HP_WC_20001118',
 'M_HP_WC_20001210',
 'M_HP_WC_20001217',
 'M_HP_WC_20010118',
 'M_HP_WC_20010208',
 'M_HP_WC_200

In [ ]:

class Competitor:
    def __init__(self, name, school, place, elo):
        self.name = name
        self.school = school
        self.place = place
        self.elo = elo

    def __eq__(self, other):
        return self.name == other.name and \
               self.school == other.school

    def __hash__(self):
        return hash((self.name, self.school))


# Used as a class for example.py, and is not relevant to the core algorithm

class Athlete:
    def __init__(self, name, school):
        self.name = name
        self.school = school

    def __eq__(self, other):
        return self.name == other.name and \
               self.school == other.school

    def __hash__(self):
        return hash((self.name, self.school))

    def __repr__(self):
        return self.name


class Meet:
    competitors = []

    def addCompetitor(self, name, place, elo, school):
        player = Competitor(name, school, place, elo)
        self.competitors.append(player)

In [ ]:
# ELO
# Background information - https://en.wikipedia.org/wiki/Elo_rating_system
# Based on this equation - http://elo-norsak.rhcloud.com/index.php

_MAXSIZE = 186
_MINBUFFEREDELO = 1900
_MAXBUFFEREDELO = 3200
_KMAX = 4.4
_BUFFEREDELOSLOPE = -.14 / 1300
_EXPODENTIALRATE = 800
_WIN = 1.0
_LOSS = 0.0

def calculateElo(players):
    N = len(players)
    # changing K multiplier depending on the size of the competition
    if N > _MAXSIZE:
        K = 1.2
    else:
        k_slope = 3 * N / 175
        K = _KMAX - k_slope
    for i in players:
        # Players with higher elo receive lower K values for stability
        multi = 1
        if i.elo > _MINBUFFEREDELO and i.elo < _MAXBUFFEREDELO:
            multi = _WIN + (_BUFFEREDELOSLOPE) * (i.elo - _MINBUFFEREDELO)
        K = multi * K
        for j in players:
            if i is not j:
                # S is the player's actual score - (1 for win) - (0 for loss)
                if i.place < j.place:
                    S = _WIN
                else:
                    S = _LOSS
                change = j.elo - i.elo
                # Expected is the expected score based off each players elo
                expected_score = 1 / \
                    (1.0 + math.pow(10.0, (change) / _EXPODENTIALRATE))
                i.elo += K * (S - expected_score)
        # Making zero be the rating system's floor (optional)
        if i.elo < 0:
            i.elo = 0

In [ ]:
import math
import csv
import json
import os
import shutil
from sys import argv
from datetime import datetime
from django.utils.encoding import smart_str
from operator import itemgetter

In [ ]:


# Elos dictionaries contain athletes keyed to an elo value
# Entries dictionaries contain athletes keyed to history of their results

elos_boys = {}
elos_girls = {}
entries_boys = {}
entries_girls = {}
_DEFELO = 1500.0


def do_elo(data, meetName, meetDate, gender):
    if gender == "female":
        elos = elos_girls
        entries = entries_girls
    elif gender == "male":
        elos = elos_boys
        entries = entries_boys

    # Add players to competition and calculate elos

    meet = Meet()
    meet.competitors = []
    for dat in data:
        name = dat[0]
        place = int(dat[1])
        school = dat[2]
        ath = Athlete(name, school)
        if ath in elos:
            elo = float(elos.get(ath))
            meet.addCompetitor(name, place, elo, school)
        else:
            # defaults to elo of 1500 on athletes first meet
            meet.addCompetitor(name, place, _DEFELO, school)
    calculateElo(meet.competitors)

    # Take results of competition and append data

    for runner in meet.competitors:
        ather = Athlete(runner.name, runner.school)
        elos[ather] = runner.elo
        if ather in entries:
            res_list = entries.get(ather)
            res_list.append([meetName, meetDate, runner.elo])
            entries[ather] = res_list
        else:
            entries[ather] = [[meetName, meetDate, runner.elo]]


In [ ]:
def align_data(filename):
    filex = open(filename)
    sort = []
    for json_string in filex:
        parsed = json.loads(json_string)
        results = parsed["results"]
        kill = False
        locs = parsed["meetLocation"]
        a_date = parsed["meetDate"]
        exact_date = datetime.strptime(a_date[0], "%A, %B %d, %Y")
        for loc in locs:
            if loc == u'Collegiate' or loc == u'MS':
                kill = True
        for result in results:
            if result.keys() == [u'maleResults'] or [u'femaleResults']:
                static = result.values()
                events = list(static)[0]
                for event in events:
                    data = []
                    data.append(exact_date)
                    data.append(parsed['meetName'])
                    if result.keys() == [u'maleResults']:
                        data.append("male")
                    elif result.keys() == [u'femaleResults']:
                        data.append("female")
                    places = []
                    details = event[u'eventDetails']
                    for detail in details:
                        killx = False
                        ath_detail_List = []
                        ath_detail_List.append(
                                        smart_str(detail[u'resultName']))
                        if detail[u'resultPlace'] == " " or \
                                detail[u'resultPlace'] == u' ':
                            killx = True
                        else:
                            ath_detail_List.append(detail[u'resultPlace'])
                        ath_detail_List.append(
                                        smart_str(detail[u'resultSchool']))
                        if killx is False:
                            places.append(ath_detail_List)
                    data.append(places)
                    if kill is False:
                        sort.append(data)
    sortx = sorted(sort, key=itemgetter(0))
    return sortx




In [ ]:
os.listdir()

['Multiplayer-elo.ipynb',
 'Cleaned data.zip',
 'HP_M.xlsx',
 'HP_W.xlsx',
 'ranking.pickle',
 'athlete.pickle',
 'Data cleaning.ipynb',
 'HP comp']

In [ ]:
filename = 'smallish.json'
filex = open(filename)

In [ ]:
  for json_string in filex:
      parsed = json.loads(json_string)

In [ ]:
for json_string in filex:
    parsed = json.loads(json_string)
    results = parsed["results"]
    print(parsed)
    print(results)

    break

In [ ]:
d = align_data(filename)

In [ ]:
d

[[datetime.datetime(2005, 9, 10, 0, 0),
  ['Yreka Elks Cross Country Invitational '],
  [['Ryland Karlovich', '6', 'Ashland'],
   ['Eric Seidlicki', '8', 'Grants Pass'],
   ['Jeremy Artoff', '30', 'Crater']]],
 [datetime.datetime(2005, 9, 10, 0, 0),
  ['Yreka Elks Cross Country Invitational '],
  [['Casey Mundell', '1', 'Crater'],
   ['Ryan Santana', '2', 'Crater'],
   ['Luke Peyralans', '4', 'Crater'],
   ['Curtis Yungen', '6', 'Crater'],
   ['Jerrod Puckett', '9', 'Crater'],
   ['Alex Miller', '50', 'Crater'],
   ['Josh Keith', '48', 'Crater'],
   ['Zane Stevens', '64', 'Crater']]],
 [datetime.datetime(2005, 9, 10, 0, 0),
  ['Yreka Elks Cross Country Invitational '],
  [['Isaac Stoutenburg', '1', 'Crater'],
   ['Jakob Keyser', '2', 'Crater'],
   ['Caleb Jacobson', '28', 'Crater'],
   ['Jon Buel', '60', 'Crater'],
   ['Scott Marshall', '63', 'Crater'],
   ['Andrew Gamble', '83', 'Crater'],
   ['Austin Carter', '104', 'Crater']]],
 [datetime.datetime(2005, 9, 10, 0, 0),
  ['Yreka Elks 

In [ ]:
d[0]

[datetime.datetime(2005, 9, 10, 0, 0),
 ['Yreka Elks Cross Country Invitational '],
 [['Ryland Karlovich', '6', 'Ashland'],
  ['Eric Seidlicki', '8', 'Grants Pass'],
  ['Jeremy Artoff', '30', 'Crater']]]

In [ ]:

def write_ath(entries):
    if entries == entries_boys:
        path = "./meets/boys"
    elif entries == entries_girls:
        path = "./meets/girls"
    if not os.path.exists("./meets/"):
        os.mkdir("./meets/")
    if not os.path.exists(path):
        os.mkdir(path + "/")
    for ath in entries:
        school_path = os.path.join(path, ath.school)
        ath_path = os.path.join(school_path, ath.name + ".csv")
        filename = "%s.csv" % ath.name
        with open((filename), "w") as fp:
            a = csv.writer(fp, delimiter=',')
            a.writerows(entries[ath])
        if os.path.exists(school_path):
            shutil.move(filename, ath_path)
        else:
            os.mkdir(school_path)
            shutil.move(filename, ath_path)


def write_elo(elos, gender):
    if gender == "male":
        name = "athlete_elo_boys.csv"
    elif gender == "female":
        name = "athlete_elo_girls.csv"
    with open((name), "w") as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(elos)




In [ ]:
def main():
    # isolates every event and pass that into the do_elo function by gender
    events = align_data(argv[1])
    count = 0
    for event in events:
        # makes sure there are 4 values of (name, date, gender, and results)
        if len(event) == 4:
            print (count)
            count += 1
            name = smart_str(event[1][0])
            date = event[0]
            gender = event[2]
            do_elo(event[3], name, date, gender)
    # sorts the dictionaries by ascending elo
    sorted_boys = sorted(elos_boys.items(), key=itemgetter(1))
    sorted_girls = sorted(elos_girls.items(), key=itemgetter(1))
    write_elo(sorted_boys, "male")
    write_elo(sorted_girls, "female")
    write_ath(entries_girls)
    write_ath(entries_boys)

FileNotFoundError: ignored

In [ ]:

# isolates every event and pass that into the do_elo function by gender
events = align_data(filename)
count = 0
for event in events:
    # makes sure there are 4 values of (name, date, gender, and results)
    if len(event) == 4:
        print (count)
        count += 1
        name = smart_str(event[1][0])
        date = event[0]
        gender = event[2]
        #do_elo(event[3], name, date, gender)
# sorts the dictionaries by ascending elo
# sorted_boys = sorted(elos_boys.items(), key=itemgetter(1))
# sorted_girls = sorted(elos_girls.items(), key=itemgetter(1))
# write_elo(sorted_boys, "male")
# write_elo(sorted_girls, "female")
# write_ath(entries_girls)
# write_ath(entries_boys)

In [ ]:

# athlete ={}
# for file in files_to_process:
#     xls = pd.ExcelFile('HP comp/'+file)

#     sheet_names = xls.sheet_names

#     for sheet in sheet_names:
#         df = xls.parse(sheet,header = 0)

#         for a in range (df.shape[0]):
#             FIS_code = df.loc[a, 'Fis Code']
#             if FIS_code not in athlete:
#                 athlete[FIS_code] = {'first_name': df.loc[a,'First Name'],
#                                     'last_name': df.loc[a, 'Last Name'],
#                                     'gender': df.loc[a, 'Gender (Races!Sb)'],
#                                     'birthdate': df.loc[a, 'Birthdate'],
#                                     'nation': df.loc[a, 'Nation'],
#                                     'competition': {df.loc[a, 'Race Date'] : {'race_id': df.loc[a,'Race ID'],
#                                                                               'location': df.loc[a, 'Location'],
#                                                                               'host_nation': df.loc[a,'Host Nation'],
#                                                                               'race_type': df.loc[a, 'Event Type'],
#                                                                               'rank': df.loc[a, 'Rank'],
#                                                                               'race_point': df.loc[a, 'Race Points']}}
#                                     }
#             else:
#                 athlete[FIS_code]['competition'][df.loc[a, 'Race Date']] = {'race_id': df.loc[a,'Race ID'],
#                                                                             'location': df.loc[a, 'Location'],
#                                                                             'host_nation': df.loc[a,'Host Nation'],
#                                                                             'race_type': df.loc[a, 'Event Type'],
#                                                                             'rank': df.loc[a, 'Rank'],
#                                                                             'race_point': df.loc[a, 'Race Points']}